In [1]:
import harmonica as hm
import xarray as xr
import rasterio
import rioxarray
from rasterio.transform import from_origin
import numpy as np
from affine import Affine

In [2]:
path = '/Users/thowe/Desktop/Quebec_MAG_DV1_GRD/Quebec_MAG_DV1_GRD.grd'

da = hm.load_oasis_montaj_grid(fname=path)

In [3]:
# Print basic info
print("Dimensions:", da.dims)
print("Shape:", da.shape)
print("Coordinates:", list(da.coords))

# Print all attributes
print("\nAttributes:")
for attr_name, attr_value in da.attrs.items():
    print(f"  {attr_name}: {attr_value}")

# Print data summary
print("\nData Summary (ignoring NaNs):")
print("  Min:", float(da.min(skipna=True)))
print("  Max:", float(da.max(skipna=True)))
print("  Mean:", float(da.mean(skipna=True)))
print("  Std Dev:", float(da.std(skipna=True)))
print("  % NaNs:", float(da.isnull().sum() / da.size * 100), "%")

Dimensions: ('northing', 'easting')
Shape: (28253, 22760)
Coordinates: ['easting', 'northing']

Attributes:
  n_bytes_per_element: 4
  sign_flag: 2
  shape_e: 22760
  shape_v: 28253
  ordering: 1
  spacing_e: 75.0
  spacing_v: 75.0
  x_origin: -876975.0
  y_origin: 13500.0
  rotation: 0.0
  base_value: 0.0
  data_factor: 1.0
  map_projection: 0
  units_x: 0
  units_y: 0
  units_z: 0
  n_valid_points: 195867186
  grid_min: -219.20159912109375
  grid_max: 363.04364013671875
  grid_median: 0.0
  grid_mean: -0.0002776627952698618
  grid_variance: 0.9875565402201316
  process_flag: 0

Data Summary (ignoring NaNs):
  Min: -219.20159912109375
  Max: 363.04364013671875
  Mean: -0.00027766279840912817
  Std Dev: 0.9937587912455751
  % NaNs: 69.5403536473754 %


In [4]:
# Rename dimensions and coordinates to standard CF-compliant names
da = da.rename({"easting": "x", "northing": "y"})

# Update coordinate axis metadata
da.coords["x"].attrs.update({
    "standard_name": "projection_x_coordinate",
    "long_name": "Easting",
    "units": "m",
    "axis": "X"
})

da.coords["y"].attrs.update({
    "standard_name": "projection_y_coordinate",
    "long_name": "Northing",
    "units": "m",
    "axis": "Y"
})

# Define the CRS with full metadata from your transform
crs = xr.DataArray(0, attrs={
    "grid_mapping_name": "lambert_conformal_conic",
    "standard_parallel": [46.0, 60.0],  # order matters: lower, then higher
    "latitude_of_projection_origin": 44.0,
    "longitude_of_central_meridian": -68.5,
    "false_easting": 0.0,
    "false_northing": 0.0,
    "semi_major_axis": 6378137.0,
    "inverse_flattening": 298.257222101,  # GRS 1980
    "epsg_code": "EPSG:6622",
    "crs_wkt": "NAD83(CSRS) / Quebec Lambert",
    "units": "m"
})

# Wrap in a dataset and link grid mapping
wrapped = xr.Dataset({"magnetics": da})
wrapped["crs"] = crs
wrapped["magnetics"].attrs["grid_mapping"] = "crs"

# Export to NetCDF
output_path = "/Users/thowe/Desktop/Quebec_MAG_DV1_GRD/Quebec_MAG_DV1_GRD.nc"
wrapped.to_netcdf(output_path)


In [5]:
# Now export the "magnetics" layer as a GeoTIFF using rioxarray

# Get the "magnetics" DataArray
mag_da = wrapped["magnetics"]

# rioxarray expects the CRS to be an attribute of the DataArray.
# If you know the EPSG code, you can write it directly:
mag_da.rio.write_crs("EPSG:6622", inplace=True)

# In many cases, rioxarray can infer the affine transform automatically,
# but you can also calculate it explicitly if needed.
#
# Assume the x and y coordinates are evenly spaced. Compute the pixel size:
x_vals = mag_da.coords["x"].values
y_vals = mag_da.coords["y"].values

# Calculate resolution along x; adjust if the coordinates do not start exactly at the cell edge.
pixel_width = (x_vals[-1] - x_vals[0]) / (len(x_vals) - 1)

# For y, note: GeoTIFFs normally use an origin at the top-left. Here we assume that y is descending,
# but if not, adjust accordingly.
pixel_height = (y_vals[0] - y_vals[-1]) / (len(y_vals) - 1)

# Create an affine transform.
# The translation offsets are adjusted so that the pixel centers are correctly represented.
transform = Affine.translation(x_vals[0] - pixel_width / 2,
                               y_vals[-1] - pixel_height / 2) * Affine.scale(pixel_width, pixel_height)

# Write the transform to the DataArray. This step is crucial if it cannot be inferred automatically.
mag_da.rio.write_transform(transform, inplace=True)

# Export the DataArray as a GeoTIFF
geotiff_output_path = "/Users/thowe/Desktop/Quebec_MAG_DV1_GRD/Quebec_MAG_DV1_GRD.tif"
mag_da.rio.to_raster(geotiff_output_path)